## bifacial_radiance examples
what to do with the new bifacial_radiance python module


#### Prerequisites (Step 0):
This software requires the previous installation of RADIANCE from https://github.com/NREL/Radiance/releases.

Make sure you add radiance to the system PATH so Python can interact with the radiance program

If you are on a PC you should also copy the Jaloxa radwinexe-5.0.a.8-win64.zip executables into `program files/radiance/bin`: http://www.jaloxa.eu/resources/radiance/radwinexe.shtml

#### STEP 1: Install and import bifacial_radiance

 - clone the bifacial_radiance repo to your local directory
 - navigate to the \bifacial_radiance directory which contains setup
 - run `pip install -e .  `  ( the period . is required, the -e flag is optional and installs in development mode where changes to the bifacial_radiance.py files are immediately incorporated into the module if you re-start the python kernel)

#### STEP 2: Move gencumulativesky.exe
Copy gencumulativesky.exe from the repo's `/bifacial_radiance/data/` directory and copy into your Radiance install directory.
This is typically found in `/program files/radiance/bin/`.  

#### STEP 3: Create a local Radiance directory for storing the scene files created
Keep scene geometry files separate from the bifacial_radiance directory.  Create a local directory somewhere that will be referenced in the next step.

#### STEP 4: Reboot the computer
This makes sure the PATH is updated

In [1]:
testfolder = r'C:\Users\cdeline\Documents\Python Scripts\TestFolder'   #point to an empty directory or existing Radiance directory

In [2]:
try:
    from bifacial_radiance import *
except ImportError:
    raise RuntimeError('bifacial_radiance is required. download distribution')

In [3]:
# Simple example system using Radiance.  We'll simulate a 1-up landscape system over a white rooftop
demo = RadianceObj('bifacial_example',testfolder)  # Create a RadianceObj 'object' named bifacial_example. no whitespace allowed

path = C:\Users\cdeline\Documents\Python Scripts\TestFolder


In [4]:
demo.setGround(0.62) # input albedo number or material name like 'concrete'.  To see options, run this without any input.

# pull in meteorological data using pyEPW for any global lat/lon
epwfile = demo.getEPW(lat = 37.5, lon = -77.6) 

# read in the weather data pulled in above. 
# If you want a different location, replace this filename with the new EPW file name in `epwfile`.    
metdata = demo.readEPW('EPWs\\USA_VA_Richmond.Intl.AP.724010_TMY.epw') 

# Solar resource definition.  Either choose a single time point, or use cumulativesky for the entire year. 
fullYear = True
if fullYear:
    demo.genCumSky(demo.epwfile) # entire year.
else:
    demo.gendaylit(metdata,4020)  # Noon, June 17th (timepoint # 4020)


Getting weather file: USA_VA_Richmond.Intl.AP.724010_TMY.epw  ... OK!
message: There were 4255 sun up hours in this climate file
Total Ibh/Lbh: 0.000000


In [5]:
# create a custom PV module type. Prism Solar Bi60. x = .984 y = 1.695. Bifaciality = 0.90
# Note: modules are currently 100% opaque.  This will be modified in the future
demo.makeModule(name='Prism Solar Bi60',x=0.984,y=1.695,bifi = 0.90)

# print available module types in data/module.json
demo.printModules()


Module Prism Solar Bi60 successfully created
Available module names: ['mini_panel', 'simple_panel', 'monopanel', 'Prism Solar Bi60']


In [9]:
# create a scene using panels in landscape at 10 deg tilt, 1.5m pitch. 0.2 m ground clearance
sceneDict = {'tilt':10,'pitch':1.5,'height':0.2,'orientation':'landscape','azimuth':180} 
module_type = 'Prism Solar Bi60'
scene = demo.makeScene(module_type,sceneDict, nMods = 20, nRows = 7) #makeScene creates a .rad file with 20 modules per row, 7 rows.
octfile = demo.makeOct(demo.getfilelist())  # makeOct combines all of the ground, sky and object files into a .oct file.
analysis = AnalysisObj(octfile, demo.basename)  # return an analysis object including the scan dimensions for back irradiance
analysis.analysis(octfile, demo.basename, scene.frontscan, scene.backscan)  # compare the back vs front irradiance  
# the frontscan and backscan include a linescan along a chord of the module, both on the front and back.  
# Return the minimum of the irradiance values, and the average of the irradiance values along a chord of the module.
print('Annual bifacial ratio: %0.3f - %0.3f' %(min(analysis.backRatio), np.mean(analysis.backRatio)) )



created bifacial_example.oct linescan in process: bifacial_example_Front
linescan in process: bifacial_example_Back
saved: results\irr_bifacial_example.csv
Annual bifacial ratio: 0.084 - 0.120


In [7]:
# Make a color render and falsecolor image of the scene.  
# Files are saved as .hdr (high definition render) files.  Try LuminanceHDR viewer (free) to view them
analysis.makeImage('side.vp')
analysis.makeFalseColor('side.vp')
# Note - if you want to have an interactive image viewer, use the `rvu` viewer - manual page here: http://radsite.lbl.gov/radiance/rvu.1.html

generating visible render of scene
generating scene in WM-2
saving scene in false color
